In [ ]:
! pip install transformers datasets tokenizers evaluate
! pip install transformers[sentencepiece]
! pip install torch
! pip install tensorflow
! pip install spacy
! pip install seqeval
! pip install ipywidgets
! pip install "ray[tune]" transformers datasets scipy scikit-learn torch

In [2]:
# mount to folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mobile_privacy/cleaned/models

Mounted at /content/drive
/content/drive/MyDrive/mobile_privacy/cleaned/models


In [16]:
# imports
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import sys, os,json
sys.path.insert(1, '/content/drive/MyDrive/mobile_privacy/cleaned')
import datasets
import transformers
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from transformers import Trainer
from transformers import TrainingArguments
from ray import tune
from seqeval.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
RANDOM_SEED = 0

In [17]:
def load_dataset(path, split=[0.9, 0.05, 0.05]):
    '''
    Load the dataset from the path as a Dataset object.
    Dataset format is:
        - id: the id of the scenario
        - tokens: tokennized words
        - ner_tags: the NER tags of the tokens
    @ param path: file path to dataset json file.
    @ param split: the [train, validation, test] split.
    '''
    with open(path, 'r') as f:
        data1 = json.load(f)
    res = {'id': [], 'tokens': [], 'ner_tags': []}
    for id, data in data1.items():
        res['id'].append(id)
        res['tokens'].append(data['words'])
        res['ner_tags'].append(data['codes'])
    dataset = datasets.Dataset.from_dict(res, features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names=[
                                'O', 'B-SIM', 'I-SIM', 'B-COM', 'I-COM', 'B-QUE', 'I-QUE'
                            ]
                        )
                    ),
                }
            ))
    if len(split) != 3 or abs(split[0] + split[1] + split[2] - 1.0) > 1e-5:
      print(split)
      raise Exception("split must be in the form [train, validation, test], and the split should sum up to 1")
    train_testvalid = dataset.shuffle(seed=RANDOM_SEED).train_test_split(test_size=split[1] + split[2], seed=RANDOM_SEED)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=split[2]/(split[1] + split[2]), seed=RANDOM_SEED)
    # gather everyone if you want to have a single DatasetDict
    train_test_valid_dataset = datasets.dataset_dict.DatasetDict(
        {
        'train': train_testvalid['train'],
        'validation': test_valid['train'],
        'test': test_valid['test']
        }
    )
    return train_test_valid_dataset

In [18]:
# tokenize and align dataset
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples, **kwargs):
    '''
    Input: a row of Dataset
    Output: use dataset.map() method to map this function for each row.
            This will tokenize each row and align the original labels.
    '''
    tokenizer = kwargs['tokenizer']
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [20]:
# execution codes
# 1. load dataset
# change DATASET_PATH to your dataset path
DATASET_PATH = '../datasets/scenarios-relabeled.json'
raw_datasets = load_dataset(DATASET_PATH, [0.8, 0.1, 0.1])
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names

# 2. auto tokenizer
model_checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    fn_kwargs={'tokenizer':tokenizer}
)

# 3. data collator and metric
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    MultiLabelBinarizer().fit_transform(true_labels)
    MultiLabelBinarizer().fit_transform(true_predictions)
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    # return all_metrics
    print(classification_report(true_labels,true_predictions ))
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# 4. define model
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

def model_init():
    return AutoModelForTokenClassification.from_pretrained(
            model_checkpoint,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
    
# 5. train
args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)
trainer = Trainer(
    # model=model,
    model_init = model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

ready


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--dslim--bert-base-NER/snapshots/f7c2808a659015eeb8828f3f809a2f1be67a2446/config.json
Model config BertConfig {
  "_name_or_path": "dslim/bert-base-NER",
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-SIM",
    "2": "I-SIM",
    "3": "B-COM",
    "4": "I-COM",
    "5": "B-QUE",
    "6": "I-QUE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-COM": 3,
    "B-QUE": 5,
    "B-SIM": 1,
    "I-COM": 4,
    "I-QUE": 6,
    "I-SIM": 2,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  

...

              precision    recall  f1-score   support

         COM       0.02      0.03      0.02        30
         QUE       0.38      0.59      0.46        44
         SIM       0.63      0.76      0.69       332

   micro avg       0.54      0.68      0.60       406
   macro avg       0.34      0.46      0.39       406
weighted avg       0.56      0.68      0.61       406



Saving model checkpoint to bert-finetuned-ner/checkpoint-270
Configuration saved in bert-finetuned-ner/checkpoint-270/config.json
Model weights saved in bert-finetuned-ner/checkpoint-270/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-270/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-270/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 30
  Batch size = 8


              precision    recall  f1-score   support

         COM       0.02      0.03      0.03        30
         QUE       0.41      0.59      0.48        44
         SIM       0.65      0.74      0.69       332

   micro avg       0.56      0.67      0.61       406
   macro avg       0.36      0.46      0.40       406
weighted avg       0.58      0.67      0.62       406



Saving model checkpoint to bert-finetuned-ner/checkpoint-300
Configuration saved in bert-finetuned-ner/checkpoint-300/config.json
Model weights saved in bert-finetuned-ner/checkpoint-300/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-300/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-300/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=300, training_loss=0.22315172831217447, metrics={'train_runtime': 6643.9327, 'train_samples_per_second': 0.361, 'train_steps_per_second': 0.045, 'total_flos': 309613907245104.0, 'train_loss': 0.22315172831217447, 'epoch': 10.0})